In [12]:
import pandas as pd
import torch
import gc
from numba import cuda
from tqdm import tqdm
import json
from datasets import Dataset
import sys
import re
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import random

from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline as transformers_pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [ ]:
#IMPORT AND PREPARE DATASET

path = ""  # Replace with your dataset path

ds = pd.read_csv(path)

ds = ds.rename(columns={'test_case': 'text', 'label_gold': 'label', 'target_ident': 'target'})
selected_columns = ['case_id', 'text', 'label', 'target','direction','functionality']
dss = ds[selected_columns]

dataset = Dataset.from_pandas(dss)
dataset = dataset.shuffle()
print(dataset)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-xl',device_map='auto')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xl')
model_name = 'google/flan-t5-xl'

In [ ]:
#IMPORT DEFINTIONS - STEP 1
def_step1 = {   "NO":"",
                "HateCheck":"Hate speech is abuse that is targeted at a protected group or at its members for being a part of that group. Protected groups are defined based on age, disability, gender identity, familial status, pregnancy, race, national or ethnic origins, religion, sex or sexual orientation, which broadly reflects international legal consensus (particularly the UK’s 2010 Equality Act, the US 1964 Civil Rights Act, and the EU’s Charter of Fundamental Rights).",
                "OL":"Hate Speech is considered any kind of content that conveys malevolent intentions toward a group or an individual.",
                "HSB":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDPC":"Hate speech is considered any kind of content that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDT":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDT":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that convey malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDPC":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDT_EDPC":"Hate speech is considered any kind of content that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDPC_EDT":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members."
                }


def_step2 = {"HSB_EDT_LAA":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics.",
            "HSB_EDT_LAA_PI":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety.",
            "HSB_EDT_LAA_Exc":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions",
            "HSB_EDT_LAA_IHS":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation.",
            "HSB_EDT_LAA_PI_Exc":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions.",
            "HSB_EDT_LAA_Exc_IHS":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions.",
            "HSB_EDT_LAA_PI_IHS":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation.",
            "HSB_EDT_LAA_PI_IHS_Exc":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions."
            }

In [ ]:
#PREPARE DATAPOINTS

model_inputs = []
labels = []
instructions = "Classify if the following text is considered hate speech or not hate speech:\n"\
              "[TEXT]\n"\
              "Please answer ONLY with 1 if it is hate speech or 0 if it is not hate speech."
        

for datapoint in tqdm(dataset):
    model_input = instructions
    model_input = model_input.replace("[TEXT]", datapoint['text'])
    if datapoint['label'] == 'hateful':
        labels.append(1)
    else:
        labels.append(0)
    model_inputs.append(model_input)
print(model_inputs[1])


In [ ]:
def metriche(predictions,labels):

    predictions = [np.nan if x == 'NaN' else x for x in predictions]

    #convert predictions to numpy array to handle NaN values
    predictions = np.array(predictions, dtype=float)  

    #replace NaN values with a default value (e.g., 0) if needed
    predictions[np.isnan(predictions)] = 0  

    precision = precision_score(labels, predictions, average='binary', zero_division=0)

    recall = recall_score(labels, predictions, average='binary', zero_division=0)

    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    
    return precision, recall, f1
    

In [ ]:
from tqdm import tqdm

def runner(model_inputs, labels, num_runs, id_def, definition, sample):
    results = []

    for j in range(num_runs):  
        if sample == 'all' or sample >= 3901:
            sample = 3901
        else:
            model_inputs = model_inputs[:sample]
            labels = labels[:sample]
    
        completions = []
        responses = []

        system_prompt = ''

        model_prompt =  "[SYSTEM_PROMPT] \n [USER_MESSAGE] \n"

        for i, inputtone in enumerate(tqdm(model_inputs)): 
            if id_def == 'NO':
                tot_in = inputtone
            else:
                tot_in = 'Given the following definition of Hate Speech: \n' + str(definition) +'\n' + inputtone 

            model_in = model_prompt.replace("[SYSTEM_PROMPT]", system_prompt)
            model_in = model_in.replace("[USER_MESSAGE]", tot_in)

            # Tokenize the input using the tokenizer
            input_ids = tokenizer(model_in, return_tensors="pt").input_ids.to("cuda")


            # Generate outputs from the model
            outputs = model.generate(input_ids,temperature=0.95, do_sample = True)

            # Decode the output IDs to text
            decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = decoded_output
            responses.append(response)


            predictions = []
            for response in responses:
                if response == '1':
                    predictions.append(1)
                elif response == '0':
                    predictions.append(0)
                else:
                    predictions.append('NaN')

        num = j+1
        print('Run number:', num, 'With definition: ', id_def)     
        numeratore = sum(1 for p,l in zip(predictions,labels) if p ==l)
        denominatore = len(predictions)
        acc = numeratore/denominatore
        print('Accuracy: ', acc)
        nans = sum(1 for p in predictions if p =='NaN')
        print('Numbers of non-answer: ',nans)
        if denominatore == nans:
            acc_no_nans = 0
        else:
            acc_no_nans = numeratore/(denominatore-nans)
        print('Accuracy without no-answers: ', acc_no_nans)

        precision, recall, f1 = metriche(predictions, labels)

        results.append({'Run': num,
                            'Model': model_name, 
                            'ID_def': id_def,
                            'Accuracy': acc, 
                            'NaNs': nans, 
                            'Precision': precision, 
                            'Recall': recall, 
                            'F1': f1 ,
                            'Input': dataset['text'],
                            'Responses': responses,
                            'Predictions': predictions, 
                            'Labels': labels, 
                            'Definition': definition,
                            'Target': dataset['target'],
                            'Direction': dataset['direction'],
                            'Functionality': dataset['functionality'] })
        
        print('Accuracy: ', acc)
        print('Precision: ', precision)
        print('Recall: ', recall)
        print('F1 Score: ', f1)
        
    return results



In [ ]:
#ALL THE DEFINITIONS
sample = 'all'  #select a number of samples or "all" for the whole dataset
num_runs = 3  #number of iterations 
step = '2' # 1 for step 1, 2 for step 2


results = []

if step == '1':
    def_dict = def_step1
    step_name = 'step1'
elif step == '2':
    def_dict = def_step2
    step_name = 'step2'

    
for k, v in def_dict.items():
    def_id = k
    definition = v
    result = runner(model_inputs, labels, num_runs=num_runs, id_def=def_id, definition=definition, sample=sample)
    results.extend(result) 

In [ ]:
#CREATE DF AND SAVE

df = pd.DataFrame(results)

csv_file_path = '../Outputs/Flant5_HC_'+ step_name +'.csv'

df.to_csv(csv_file_path, index=False)  